# Análise de Músicas do Spotify

In [1]:
import pandas as pd
import requests
import spotipy
import re

In [2]:
credentials = spotipy.oauth2.SpotifyClientCredentials()
spotify = spotipy.Spotify(client_credentials_manager=credentials)

In [3]:
def get_uri_from_link(link: str) -> str:
    """Extrai o URI de uma playlist do link dela"""

    uri = re.search(r"\/.{22}\?", link)

    try:
        uri = uri.group()[1:-1]
    
    except AttributeError:
        print(f"Não há URI para o link passado: {link}")

    return uri

def get_artists_from_playlist(playlist_uri: str) -> dict:
    """Extrai os artistas de uma playlist e os guarda em um dicionario para uso posterior
    
    playlist_uri: Playlist que sera analisada
    return: Dict("URI": ["URI Artista"], "NOME": ["Nome do Artista"])
    """

    artists = {"uri": [], "name": []}
    playlist_tracks = spotify.playlist_tracks(playlist_id=playlist_uri)

    for track_data in playlist_tracks['items']:

        if track_data['track']:

            artists["uri"] = artists["uri"] + [track_data['track']['artists'][0]['uri']]
            artists["name"] = artists["name"] + [track_data['track']['artists'][0]['name']]

    return artists

def get_tracks_from_playlist(playlist_uri: str) -> dict:
    """Extrai as musicas de uma playlist e os guarda em um dicionario para uso posterior
    
    playlist_uri: Playlist que sera analisada
    return: Dict("URI_ARTISTA": ["URI Artista"], "URI_MUSICA": ["URI da Musica"], "NOME_MUSICA": ["Nome da Musica"], "POPULARIDADE": ["Popularidade da Musica (X - X)"], "DURACAO": ["Duracao da Musica (s)"])
    """
    pass

In [9]:
playlist_uri = get_uri_from_link("https://open.spotify.com/playlist/4M8zkNUMOu03nU8mywhKkj?si=4bc57060ef344466")
playlist_tracks = spotify.playlist_tracks(playlist_id=playlist_uri)


In [5]:
playlist_tracks = spotify.playlist_tracks(playlist_id=playlist_uri, fields=['items'])
playlist_tracks

{'items': [{'added_at': '2022-05-10T01:25:36Z',
   'added_by': {'external_urls': {'spotify': 'https://open.spotify.com/user/vbaldoino'},
    'href': 'https://api.spotify.com/v1/users/vbaldoino',
    'id': 'vbaldoino',
    'type': 'user',
    'uri': 'spotify:user:vbaldoino'},
   'is_local': False,
   'primary_color': None,
   'track': {'album': {'album_type': 'single',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/6gezkje7GoJlQbHBgLXHuu'},
       'href': 'https://api.spotify.com/v1/artists/6gezkje7GoJlQbHBgLXHuu',
       'id': '6gezkje7GoJlQbHBgLXHuu',
       'name': 'Destiny Rogers',
       'type': 'artist',
       'uri': 'spotify:artist:6gezkje7GoJlQbHBgLXHuu'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'B

In [14]:
print(playlist_tracks['items'][0]['track']['artists'][0]['name'])
print(playlist_tracks['items'][0]['track']['duration_ms'])
print(playlist_tracks['items'][0]['track']['popularity'])

Destiny Rogers
203445
69


In [10]:
import pprint

pp = pprint.PrettyPrinter(indent=4)
pp.pprint(playlist_tracks)

{   'href': 'https://api.spotify.com/v1/playlists/4M8zkNUMOu03nU8mywhKkj/tracks?offset=0&limit=100&additional_types=track',
    'items': [   {   'added_at': '2022-05-10T01:25:36Z',
                     'added_by': {   'external_urls': {   'spotify': 'https://open.spotify.com/user/vbaldoino'},
                                     'href': 'https://api.spotify.com/v1/users/vbaldoino',
                                     'id': 'vbaldoino',
                                     'type': 'user',
                                     'uri': 'spotify:user:vbaldoino'},
                     'is_local': False,
                     'primary_color': None,
                     'track': {   'album': {   'album_type': 'single',
                                               'artists': [   {   'external_urls': {   'spotify': 'https://open.spotify.com/artist/6gezkje7GoJlQbHBgLXHuu'},
                                                                  'href': 'https://api.spotify.com/v1/artists/6gezkje7GoJlQ